In [ ]:
%matplotlib inline

# Grid Generation on Climatology

As a first step in coordinate development, we'll work on sections of climatology from [WOA13](https://www.nodc.noaa.gov/OC5/woa13/). Because temperature is given in-situ, we first have to convert to potential temperature. Similar to [convert_WOA13](https://github.com/adcroft/convert_WOA13), we use the Python `gsw` package, which implements TEOS-10.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d
import hycom

As an initial set of input files, we load up THREDDS URLs for monthly averaged climatologies from 2005-2012.

In [ ]:
temp_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/decav/1.00/woa13_decav_t00_01v2.nc'
salt_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/decav/1.00/woa13_decav_s00_01v2.nc'

temp_urls_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/A5B2/0.25/woa13_A5B2_t00_04v2.nc'
salt_urls_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/A5B2/0.25/woa13_A5B2_s00_04v2.nc'

# Sections

Although our choice of coordinate should apply globally, we're particularly interested in a few troublesome spots, where there tend to always be problems, such as the Denmark Strait and the Sulu Sea. We may also care about dense overflows off Antarctica.

## Atlantic
We have an Atlantic section at around 25 degrees west, which should include a portion of Denmark Strait overflow. We'll start with state data from the 1 degree WOA13 dataset.

In [ ]:
temp = Dataset(temp_url, 'r')
salt = Dataset(salt_url, 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

In [ ]:
lon_w = -25.5
#lon_i = (lon_s <= lon) & (lon <= lon_e)

t_sect = temp.variables['t_an'][0,:,:,lon==lon_w].squeeze()
s_sect = salt.variables['s_an'][0,:,:,lon==lon_w].squeeze()

We want to take an contiguous latitude section, so we'll remove Greenland and everything north of it, as well as Antarctica. There's also a weird masked column at around 35 which is probably an island...

In [ ]:
# empty columns are entirely masked
empty = np.sum(~t_sect.mask, axis=0) == 0
empty[169:] = True # mask above Greenland

In [ ]:
lat = lat[~empty]
t_sect = t_sect[:,~empty]
s_sect = s_sect[:,~empty]

Using TEOS-10, we can convert from practical salinity to absolute salinity, and from in-situ temperature to conservative temperature. From here, we can compute the locally-referenced density, and the potential density referenced to 2000m.

In [ ]:
sa_sect = np.empty_like(s_sect)
ct_sect = np.empty_like(t_sect)
rho_sect = np.empty_like(s_sect)
rhop_sect = np.empty_like(rho_sect)

for i in range(s_sect.shape[1]):
    sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, lon_w, lat[i])
    ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)
    rho_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], dep)
    rhop_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], 2000)

Let's take a look at the two density sections, the locally-refenced density and the potential density referenced to 2000m.

In [ ]:
ax = plt.subplot(211)
plt.pcolormesh(lat, dep, rho_sect)
ax.invert_yaxis()
#plt.colorbar()

ax = plt.subplot(212)
plt.pcolormesh(lat, dep, rhop_sect)
ax.invert_yaxis()
#plt.colorbar()

# Observation Grid

We have observations at particular depth levels, which are given by masking the depth coordinate by the mask from a particular column of our temp/salt data. We assume these observations are at interfaces (because we have an observation at z=0, for example), and therefore we calculate layer averages from these so that we can perform remapping later. We also store the actual depth of each column (we don't have partial-cell topography available though) so that we can ensure regularity of our interpolated grid.

In [ ]:
sa_int = sa_sect
ct_int = ct_sect

In [ ]:
sa_lay = (sa_int[1:,:] + sa_int[:-1,:]) / 2
ct_lay = (ct_int[1:,:] + ct_int[:-1,:]) / 2

In [ ]:
# depths of all interfaces on which observations are present
gr_int = np.ma.array(np.tile(dep.reshape(-1, 1), (1, sa_int.shape[1])), mask=sa_int.mask)

# thicknesses of all layers between interfaces
gr_th  = np.diff(gr_int, axis=0)

# bottom interface at each column
topo = gr_int.max(axis=0).compressed()

# Remapping

Now we can define a function that will remap from our source data to any target grid.

**Note:** *We might want to do this for temp/salt rather than conservative temp and absolute salt?*

In [ ]:
remap_cs = mom_remapping.Remapping_Cs()
remap_cs.remapping_scheme = 4 # PQM_IH4IH3
remap_cs.degree = 4

In [ ]:
def remap(h):
    """
    Remap from original climatological grid according to h
    """

    sa_remap = np.empty_like(h)
    ct_remap = np.empty_like(h)

    # remap by columns
    for i in range(h.shape[1]):
        # we need to make sure we deal with unmasking here,
        # otherwise we'll get the fill values for thickness
        # and salt/temp, which would be just a little weird
        sa_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       sa_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        ct_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       ct_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        
    return sa_remap, ct_remap

# Grid Generation

We can go about generating the grid using the Hofmeister et al. (2010) technique. This involves a vertical grid diffusion to optimise for buoyancy, shear, near-surface zooming and a background component. Then we're left with an isopycnal or neutral density curvature tendency term.

## Neutral Density Curvature

First, we'll define a function to calculate the neutral density curvature `ndc` at interfaces, which is where we need to calculate the tendency term.

In [ ]:
def ndc_int(z_int, sa_lay, ct_lay):
    """
    Calculate neutral density curvature between
    adjacent columns, given their (absolute) salinity, (conservative)
    temperature and physical positions (or pressure).
    
    z_int gives the current location of all model interfaces,
    and sa_lay and ct_lay give the absolute salinity and conservative
    temperature cell mean values between these interfaces.
    """
    
    # first, calculate layer thicknesses
    h = np.diff(z_int, axis=0)
    
    # use the layer thicknesses to interpolate interface values
    # using weighted average of the cell mean values on either side
    # XXX is this the right thing to do?
    sa_int = (sa_lay[1:,:] * h[1:,:] + sa_lay[:-1,:] * h[:-1,:]) / (h[1:,:] + h[:-1,:])
    ct_int = (ct_lay[1:,:] * h[1:,:] + ct_lay[:-1,:] * h[:-1,:]) / (h[1:,:] + h[:-1,:])
    
    # drop top and bottom interfaces, since we won't calculate
    # the curvature at either of these places (those interfaces
    # can't move)
    z_int = z_int[1:-1]
    
    # for dealing with edges, extend data with a ghost column
    # to give a Neumann boundary condition (dC/dx = 0)
    zi_gst = np.concatenate([ z_int[:,[0]],  z_int,  z_int[:,[-1]]], axis=1)
    sa_gst = np.concatenate([sa_int[:,[0]], sa_int, sa_int[:,[-1]]], axis=1)
    ct_gst = np.concatenate([ct_int[:,[0]], ct_int, ct_int[:,[-1]]], axis=1)
    
    rl, al, bl = gsw.rho_alpha_beta((sa_gst[:,1:-1] + sa_gst[:,:-2]) / 2,
                                    (ct_gst[:,1:-1] + ct_gst[:,:-2]) / 2,
                                    (zi_gst[:,1:-1] + zi_gst[:,:-2]) / 2)
    rr, ar, br = gsw.rho_alpha_beta((sa_gst[:,1:-1] + sa_gst[:,2:]) / 2,
                                    (ct_gst[:,1:-1] + ct_gst[:,:-2]) / 2,
                                    (zi_gst[:,1:-1] + zi_gst[:,:-2]) / 2)
    
    ndd_l = rl * (bl * (sa_gst[:,:-2] - sa_gst[:,1:-1])
                 -al * (ct_gst[:,:-2] - ct_gst[:,1:-1]))
    ndd_r = rr * (br * (sa_gst[:,2:]  - sa_gst[:,1:-1])
                 -ar * (ct_gst[:,2:]  - ct_gst[:,1:-1]))

    return ndd_l + ndd_r

## Example

Let's take a look at these neutral density differences `ndc`, on the uniform grid 50 level grid remapped from the source data. First we show the original potential density field from the remapped data.

In [ ]:
n = 50
# uniform thickness from 0 to max topo with n layers
h = np.ones((n,lat.size)) * topo.max() / n

h_i = np.where(h.cumsum(axis=0) > topo)
# deflate all layers below topography
h[h_i] = 1e-3
# get unique latitudes (to get the first cell that needs inflation)
_, i = np.unique(h_i[1], return_index=True)
# inflate
h[h_i[0][i], h_i[1][i]] += (topo - h.sum(axis=0))[h_i[1][i]]

s, t = remap(h)

plt.pcolormesh(gsw.rho(s, t, 2000))
plt.gca().invert_yaxis()
plt.colorbar()

Now we actually calculate the curvature and print some statistics to see what kind of numbers we're dealing with.

In [ ]:
z = np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)
ndc = ndc_int(z, s, t)

print('min:\t\t{}\nmax:\t\t{}\nmean:\t\t{}\nmean (abs):\t{}\nsd:\t\t{}\nsd (abs):\t{}'.format(
    ndc.min(), ndc.max(), ndc.mean(), np.abs(ndc).mean(), ndc.std(), np.abs(ndc).std()))

In [ ]:
plt.figure(figsize=(10,6))
plt.pcolormesh(lat, z[1:-1], np.log10(np.abs(ndc)))
plt.gca().invert_yaxis()
plt.colorbar()

## Diffusing interfaces

Now we define a function to compute the optimised interface positions according to the diffusivitye quation. For the moment, we've only included the surface zooming, buoyancy and background terms. This is because we don't have any velocity data from which to calculate the shear term. We could however calculate the shear term from thermal wind if required.

In [ ]:
def diffuse(z_int, c_surf, d_surf, c_n2, dt, t_grid=3600, d_rho=0.5):
    """
    Use an implicit diffusivity equation to evolve the grid defined by z_int at timestep dt.
    """
    
    # allocate new grid
    z_next = np.empty_like(z_int)
    z_lay = (z_int[1:,:] + z_int[:-1,:]) / 2
    
    # gravity used by gsw
    g = 9.7963
    
    I = z_int.shape[0] - 1 # number of layers
    A = np.zeros((3, I+1)) # diffusion system coefficients

    # iterate over columns
    for i in range(z_int.shape[1]):
        # calculate local buoyancy term at interfaces
        # from temp/salt data at the centre of layers
        n2, z_c = gsw.Nsquared(sa[:,i], ct[:,i], z_lay[:,i])
        # drho_dz term to convert to distance
        dz_r = np.maximum((n2 * 1e4) / g**2, 0)
        # diffusivity coefficient on interfaces
        k_n2_int = dz_r / d_rho
        # interpolate the diffusivity coefficient from interfaces
        # to layers, where they apply in the diffusion equation
        # we have to (linearly) extrapolate into the top and
        # bottom layers, because we don't have the buoyancy frequency at
        # the surface or the very bottom
        f = interp1d(z_c, k_n2_int, bounds_error=False, fill_value="extrapolate")
        k_n2 = f(z_lay[:,i])
        
        # determine total grid coefficient from
        # background term 1/D, where D is local depth
        # surface stretching 1/(d + d_0) for distance
        # from surface d, and factor d_0
        #
        # k_grid = D/tgrid * (c_surf*k_surf + c_n2*k_n2 + c_b*k_b)
        k_grid = (z_int[-1,i] * (c_surf / (d_surf + np.maximum(z_lay[:,i], 0)) + \
                                 c_n2 * k_n2) \
                  + (1 - c_n2 - c_surf)) / t_grid

        # fill in implicit system coefficients
        A[0,2:]     = -dt * I**2 * k_grid[1:]
        A[2,:-2]    = -dt * I**2 * k_grid[:-1]
        A[1,[0,-1]] = 1 # boundary conditions
        A[1,1:-1]   = 1 + I**2 * dt * (k_grid[1:] + k_grid[:-1])

        # solve tridiagonal system
        z_next[:,i] = solve_banded((1, 1), A, z_int[:,i])
        
    return z_next

## Neutral Density Curvature Tendency

Instead of including the adaptation to neutral density curvature into the diffusion equation, we can implement it in a separate step, where the neutral density curvature is converted to a tendency, depending on the local stratification.

In [ ]:
def ndc_tendency(z_int, sa, ct):
    """
    Calculate interfacial tendency according to
    the neutral density curvature, calculated on
    interfaces.
    """
    
    # gravity used by gsw
    g = 9.7963
    
    # calculate neutral density curvature on the interfaces
    ndc = ndc_int(z_int, sa, ct)
    
    # calculate local buoyancy frequency on interfaces to get convert from
    # density differences to an interfacial displacement
    n2 = gsw.Nsquared(sa, ct, (z_int[1:,:] + z_int[:-1,:]) / 2)[0]
    # set a minimum value of n2 so we don't divide by zero
    n2 = np.maximum(n2, 1e-10)
    dz = (g**2 * ndc) / (1e4 * n2)

    # maximum interface movement is limited by half layer thickness in
    # the direction the interface is moving
    """
    h = np.diff(z_int, axis=0)
    h_i = np.arange(dz.shape[0])[:,np.newaxis] + (np.sign(dz) / 2 + 0.5).astype(int)
    h_j = np.tile(np.arange(h_i.shape[1])[np.newaxis,:], (h_i.shape[0], 1))
    h_dz = h[h_i,h_j]
    """
    h_dz = np.diff((z_int[1:,:] + z_int[:-1,:]) / 2, axis=0)
    dz = np.sign(dz) * np.minimum(np.abs(dz), h_dz / 2)
    
    return dz

## Target Isopycnal Tendency

Here we can simply calculate a tendency based on a list of target densities (of interfaces).

In [ ]:
def iso_tendency(z_int, sa, ct, targs):
    g = 9.7963

    # calculate current densities of layers
    s2 = gsw.rho(sa, ct, 2000)
    
    # interpolate (linearly) onto interfaces by column
    s2_int = np.empty_like(z_int[1:-1,:])
    for i in range(s2_int.shape[1]):
        f = interp1d((z_int[1:,i] + z_int[:-1,i]) / 2, s2[:,i], bounds_error=False, fill_value='extrapolate')
        s2_int[:,i] = f(z_int[1:-1,i])
        
    s2_diff = targs[1:-1,np.newaxis] - s2_int
        
    n2 = gsw.Nsquared(sa, ct, (z_int[1:,:] + z_int[:-1,:]) / 2)[0]
    # set a minimum value of n2 so we don't divide by zero
    n2 = np.maximum(n2, 1e-10)
    dz = (g**2 * s2_diff) / (1e4 * n2)

    # maximum interface movement is limited by half layer thickness in
    # the direction the interface is moving
    h = np.diff(z_int, axis=0)
    h_i = np.arange(dz.shape[0])[:,np.newaxis] + (np.sign(dz) / 2 + 0.5).astype(int)
    h_j = np.tile(np.arange(h_i.shape[1])[np.newaxis,:], (h_i.shape[0], 1))
    h_dz = h[h_i,h_j]
    dz = np.sign(dz) * np.minimum(np.abs(dz), h_dz / 2)
    
    return dz

## Generation Algorithm

Now we have the generation algorithm as follows:

- move interfaces according to their isopycnal/neutral density curvature tendency (3D term)
- enforce grid regularity
  - minimum thickess of all layers
  - conservation of total column thickness
- optimise interfaces according to diffusion equation, based on e.g. buoyancy or distance from surface

In [ ]:
def generate(z, sa, ct, dt, alpha=0.5, isopycnal=False, isos=None, t_grid=3600*3, c_surf=0, d_surf=0, c_n2=0):
    """
    Return a new grid specified by interface positions
    for a current grid given by interface positions
    as well as the density variables and an optimisation
    timestep.
    
    Parameter alpha determines the amount of tendency used
    to move interfaces in the 3D step (based on neutral
    density curvature or target isopycnals).
    """
    
    z_new = z.copy()
    
    if isopycnal:
        # calculate grid tendency from target isopycnals
        dz_tend = iso_tendency(z, sa, ct, isos)
    else:
        # calculate grid tendency term from neutral density curvature
        dz_tend = ndc_tendency(z, sa, ct)
    
    # apply tendency to interior interfaces
    z_new[1:-1] += alpha * dz_tend
    
    # calculate new layer thickness and
    # enforce minimum layer thickness and positive depths
    h = np.diff(z_new, axis=0)
    # 1mm thick minimum
    h = np.maximum(h, 1e-3)
    
    # reinflate each column
    # weight layers through the water column, for example
    # to give thinner surface layers
    w = 1 # uniform weighting
    h *= w
    # use actual bottom depths
    h *= topo / np.sum(h, axis=0)
    
    # check that we didn't introduce any negative thickness layers
    if np.any(h < 0):
        print('negative thickness at iteration', k)
    
    # recalculate z from h
    z_new = np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)
    
    # check that the total column thickness is preserved
    if np.any(np.abs(z_new[-1,:] - topo) > 1e-10):
        print('bottom moved at iteration', k)
        print(z_new[-1,:] - topo)
    
    # optimise layers by diffusion
    return diffuse(z_new, c_surf=c_surf, d_surf=d_surf, c_n2=c_n2, dt=100, t_grid=t_grid, d_rho=0.5)

## Testing generation algorithm

Let's test the generation algorithm in a few cases. The first is to use the neutral density curvature as a tendency term for adjusting the grid, and the second is to move the grid toward target isopycnals. In both cases, we start with a 50-layer sigma grid (i.e. uniform thickness layers, distributed between the surface and local topography).

In [ ]:
n = 50
# uniform thickness from 0 to max topo with n layers
h = np.ones((n,lat.size)) * topo.max() / n

h_i = np.where(h.cumsum(axis=0) > topo)
# deflate all layers below topography
h[h_i] = 1e-3
# get unique latitudes (to get the first cell that needs inflation)
_, i = np.unique(h_i[1], return_index=True)
# inflate
h[h_i[0][i], h_i[1][i]] += (topo - h.sum(axis=0))[h_i[1][i]]

sa, ct = remap(h)

# save initial neutral density curvature and density to see
# how things change after iteration (see above for plots of these
# quantities)
ndc_init  = ndc_int(z, sa, ct)
rhop_init = gsw.rho(sa, ct, 2000)

In [ ]:
plt.pcolormesh(lat, z, h)
plt.plot(lat, z.T, 'k', linewidth=0.4)
plt.colorbar()
plt.gca().invert_yaxis()

### Neutral density curvature generation

First we'll use the neutral density curvature, along with the regular grid diffusion.

In [ ]:
z_ndc = z.copy()
sa_ndc = sa
ct_ndc = ct

for i in range(10):
    z_ndc = generate(z_ndc, sa_ndc, ct_ndc,
                     alpha=0.25, dt=10, t_grid=3600*24,
                     c_surf=0.1, d_surf=200)
    h_ndc = np.diff(z_ndc, axis=0)
    sa_ndc, ct_ndc = remap(h_ndc)

In [ ]:
plt.figure(figsize=(10,6))
plt.pcolormesh(lat, z_ndc, h_ndc)
plt.plot(lat, z_ndc.T, 'w', linewidth=0.6)
plt.colorbar()
plt.title('layer thickness after ndc opt.')
plt.gca().invert_yaxis()

In [ ]:
ndc_final = ndc_int(z_ndc, sa_ndc, ct_ndc)

plt.figure(figsize=(10, 6))

ax = plt.subplot(211)
plt.pcolormesh(lat, z_ndc[1:-1,:], np.log10(np.abs(ndc_final)))
plt.colorbar()
plt.title('log10(ndc) after opt.')
ax.xaxis.set_ticklabels([])
ax.invert_yaxis()

ax = plt.subplot(212)
plt.pcolormesh(lat, z_ndc[1:-1,:], np.log10(np.abs(ndc_final - ndc_init)))
plt.colorbar()
plt.title('log10(ndc_f - ndc_i)')
ax.invert_yaxis()

In [ ]:
rhop_final = gsw.rho(sa_ndc, ct_ndc, 2000)

plt.figure(figsize=(10, 6))

ax = plt.subplot(211)
plt.pcolormesh(lat, z_ndc, rhop_final)
plt.plot(lat, z_ndc.T, 'k', linewidth=0.5)
plt.colorbar()
plt.title('potential density after ndc opt.')
ax.invert_yaxis()
ax.xaxis.set_ticklabels([])

ax = plt.subplot(212)
plt.pcolormesh(lat, z_ndc, rhop_final - rhop_init)
#plt.plot(lat, z_ndc.T, 'k', linewidth=0.5)
plt.title('potential density difference from initial')
plt.colorbar()
ax.invert_yaxis()

### Target isopycnal Generation

We can use a list of target densities (from those actually used in MOM6) instead of a reduction in neutral density curvature.

In [ ]:
target_dens = np.array([1010, 1016.1289, 1020.843, 1024.821, 1027.0275, 1028.2911, 
    1029.2795, 1030.1194, 1030.8626, 1031.5364, 1032.1572, 1032.7358, 
    1033.2798, 1033.7948, 1034.2519, 1034.5828, 1034.8508, 1035.0821, 
    1035.2886, 1035.4769, 1035.6511, 1035.814, 1035.9675, 1036.1107, 
    1036.2411, 1036.3615, 1036.4739, 1036.5797, 1036.68, 1036.7755, 
    1036.8526, 1036.9024, 1036.9418, 1036.9754, 1037.0052, 1037.0323, 
    1037.0574, 1037.082, 1037.1066, 1037.1312, 1037.1558, 1037.1804, 
    1037.206, 1037.2337, 1037.2642, 1037.2986, 1037.3389, 1037.3901, 
    1037.475, 1037.7204, 1038])

In [ ]:
z_iso = z.copy()
sa_iso = sa
ct_iso = ct

for i in range(10):
    z_iso = generate(z_iso, sa_iso, ct_iso, alpha=0.5,
                     dt=100, isopycnal=True, isos=target_dens, t_grid=1e6)
    h_iso = np.diff(z_iso, axis=0)
    sa_iso, ct_iso = remap(h_iso)

In [ ]:
plt.pcolormesh(lat, z_iso, h_iso)
plt.plot(lat, z_iso.T, 'w', linewidth=0.5)
plt.colorbar()
plt.title('layer thickness after iso opt.')
plt.gca().invert_yaxis()

In [ ]:
rhop_final = gsw.rho(sa_iso, ct_iso, 2000)

plt.figure(figsize=(10, 6))

ax = plt.subplot(211)
plt.pcolormesh(lat, z_iso, rhop_final)
plt.plot(lat, z_iso.T, 'k', linewidth=0.5)
plt.colorbar()
plt.title('potential density after ndc opt.')
ax.invert_yaxis()
ax.xaxis.set_ticklabels([])

ax = plt.subplot(212)
plt.pcolormesh(lat, z_iso, rhop_final - rhop_init)
#plt.plot(lat, z_iso.T, 'k', linewidth=0.5)
plt.title('potential density difference from initial')
plt.colorbar()
ax.invert_yaxis()

# Replicating the MOM6 HyCOM grid

As a point of comparison, we'll implement MOM6's 75-level HyCOM grid on our climatology section, which will hopefully let us see what's going wrong with it, and whether any of our ideas can help.

Above, we used the 50-level target densities from the global_ALE experiment. Now we'll use the 75-level targets from OM4_025. The depth coordinates are generated by the string `FNC1:2,4000,4.5,.01`.

In [ ]:
s2_75 = [1010, 1014.3034, 1017.8088, 1020.843, 1023.5566, 1025.813, 
    1027.0275, 1027.9114, 1028.6422, 1029.2795, 1029.852, 1030.3762, 
    1030.8626, 1031.3183, 1031.7486, 1032.1572, 1032.5471, 1032.9207, 
    1033.2798, 1033.6261, 1033.9608, 1034.2519, 1034.4817, 1034.6774, 
    1034.8508, 1035.0082, 1035.1533, 1035.2886, 1035.4159, 1035.5364, 
    1035.6511, 1035.7608, 1035.8661, 1035.9675, 1036.0645, 1036.1554, 
    1036.2411, 1036.3223, 1036.3998, 1036.4739, 1036.5451, 1036.6137, 
    1036.68, 1036.7441, 1036.8062, 1036.8526, 1036.8874, 1036.9164, 
    1036.9418, 1036.9647, 1036.9857, 1037.0052, 1037.0236, 1037.0409, 
    1037.0574, 1037.0738, 1037.0902, 1037.1066, 1037.123, 1037.1394, 
    1037.1558, 1037.1722, 1037.1887, 1037.206, 1037.2241, 1037.2435, 
    1037.2642, 1037.2866, 1037.3112, 1037.3389, 1037.3713, 1037.4118, 
    1037.475, 1037.6332, 1037.8104, 1038]

In [ ]:
def dz_f1(n, dz_min, total, power, precision):
    dz = np.empty(n)
    
    # initial profile
    for i in range(n):
        dz[i] = (i / (n - 1)) ** power
    
    # rescale to total depth and round to precision
    dz[:] = (total - n*dz_min) * (dz[:] / np.sum(dz))
    dz[:] = np.around(dz[:], decimals=precision)
    
    # adjust bottom
    dz[-1] += total - np.sum(dz[:] + dz_min)
    dz[-1] = np.around(dz[-1], decimals=precision)
    
    dz[:] += dz_min
    
    return dz

Calculate nominal grid spacings and adjust to local topography.

In [ ]:
dz_75 = dz_f1(75, 2, 4000, 4.5, 2)
z_75 = np.insert(dz_75.cumsum(), 0, 0)[:,np.newaxis]

z_75_full = np.tile(z_75, (1, sa_sect.shape[1]))

np.putmask(z_75_full, z_75_full > topo, topo)

Calculate maximum interface depths and maximum layer thicknesses, also from the `dz_f1()` generating function.

In [ ]:
max_int_depth = np.insert(dz_f1(75, 5, 8000, 1, 2).cumsum(), 0, 0)
max_lay_thick = dz_f1(75, 400, 31000, 0.1, 2)

Translate to layer thicknesses and remap our climatology.

In [ ]:
h = np.diff(z_75_full, axis=0)
#h = np.maximum(h, 1e-3)

sa_map, ct_map = remap(h)

In [ ]:
z_new, z_bnd = \
    hycom.hycom(h, sa_map, ct_map, s2_75, dz_75, max_int_depth, max_lay_thick)

## HyCOM Grids

In the first plot, we're showing the HyCOM grid without any adjustment to the isopycnal positions. In the second plot, we show the grid after the three kinds of adjustment: any interface which is shallower than its nominal position is moved to this nominal position, but interfaces can't be deeper than a specified depth. Additionally, layers can't be thicker than a specified thickness.

In [ ]:
plt.pcolormesh(lat, z_new, gsw.rho(sa_map, ct_map, 2000))
plt.gca().invert_yaxis()
plt.colorbar()
plt.plot(lat, z_new.T, 'k', linewidth=0.5);

In [ ]:
plt.pcolormesh(lat, z_bnd, gsw.rho(sa_map, ct_map, 2000))
plt.gca().invert_yaxis()
plt.plot(lat, z_bnd.T, 'k', linewidth=0.5);